# Missing person and vehicle theft are the most common crimes without arrest resolution in San Francisco

The considered data set consists of crime incidents that have happened during the summer of 2014 in San Francisco. During this analysis, I would like to answer a few questions:
- How many crimes were arrest resolution?
- Which of the most common crimes have poor percentage of positive resolutions (arrests)?
- Is there any relation between day of week and number of crimes?

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ds = pd.read_csv("data/sanfrancisco_incidents_summer_2014.csv", parse_dates=["Date"])

## Crimes and arrests number per day

In [ ]:
crimes_per_day = ds.pivot_table("IncidntNum", aggfunc="count", index='Date')

arrests = ds[(ds.Resolution == 'ARREST, BOOKED') | (ds.Resolution == 'ARREST, CITED')]
arrests_per_day = arrests.pivot_table("IncidntNum", aggfunc='count', index='Date')

fig, ax = plt.subplots(2, 1, figsize=(16, 8), sharex=True)
fig.subplots_adjust(hspace=0.1)

color_cycle = plt.rcParams['axes.color_cycle']
with pd.plot_params.use('x_compat', True):
    crimes_per_day.plot(ax=ax[0], title="Number of crimes", ylim=(200, 450), color=color_cycle[0])
    arrests_per_day.plot(ax=ax[1], title="Number of arrests", ylim=(0, 150), color=color_cycle[1])

fig.savefig('figs/daily_trend.png', bbox_inches='tight')

On the following chart, the number of crimes and the number of arrests per day was presented.

![Alt text](files/figs/daily_trend.png)
<small><i><a href="SanFranciscoDataAnalysis.ipynb#Crimes-and-arrests-number-per-day">figure source</a></i></small>

As it can be noted, the number of arrests seems to be correlated with the number of crimes. On the other hand there are also days, when the number of crimes increases suddenly, but number of arrests does not change significantly. It may suggests that for some cases, the effectiveness in punishing criminals decreases.

## Crimes number for top 10 the most common categories

In [ ]:
crimes_per_cat = ds.pivot_table("IncidntNum", aggfunc="count", index='Category')
arrests_per_cat = arrests.pivot_table("IncidntNum", aggfunc='count', index='Category')

per_cat = pd.DataFrame({"crimes_per_cat": crimes_per_cat, "arrests_per_cat": arrests_per_cat})
per_cat = per_cat.fillna(0)
per_cat = per_cat.sort('crimes_per_cat', ascending=False).head(10)
per_cat["arrests_perc"] = per_cat.arrests_per_cat * 100 / per_cat.crimes_per_cat
per_cat = per_cat.sort('arrests_perc', ascending=True)
per_cat = per_cat.drop('crimes_per_cat', 1)
per_cat = per_cat.drop('arrests_per_cat', 1)

ax = per_cat.plot(kind="bar", alpha=0.7)
ax.legend().set_visible(False)

plt.xlabel('Top 10 common crimes')
plt.ylabel('Percentage of arrests [%]')
plt.savefig('figs/arrests_perc_per_cat.png', bbox_inches='tight')

In order to find out which of the most common crimes categories have the smallest percentage of arrests, the percentage of arrests for the top 10 most common crimes (with the larger number of crimes) was presented on the following figure.

![Alt text](files/figs/arrests_perc_per_cat.png)
<small><i><a href="SanFranciscoDataAnalysis.ipynb#Crimes-number-for-10-top-common-categories">figure source</a></i></small>

What is very interesting on this figure is that missing person and vehicle theft - crimes which seems to be very 
serious - have very small of percentage of arrests. On the other hand other serious crimes like assault or drug/narcotic are much more often resolve with arrests.

## Number of arrests per week for missing person and vehicle theft

In [ ]:
top_cat = ds[(ds.Category == 'MISSING PERSON') | (ds.Category == 'VEHICLE THEFT')]

crimes_per_day_of_week = top_cat.pivot_table("IncidntNum", columns=["Category"], aggfunc="count", index='DayOfWeek')
crimes_per_day_of_week = crimes_per_day_of_week.reindex(index = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

ax = pd.DataFrame(crimes_per_day_of_week).plot(kind="bar", alpha=0.7)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)

plt.xlabel('Day of week')
plt.ylabel('Number of arrests')

plt.savefig('figs/top_crimes_per_week.png', bbox_inches='tight')

On the last figure, the number of arrests per week was presented for missing person and vehicle theft categories.

![Alt text](files/figs/top_crimes_per_week.png)
<small><i><a href="SanFranciscoDataAnalysis.ipynb#Number-of-arrests-per-week-for-missing-person-and-vehicle-theft">figure source</a></i></small>

As it can be seen, the number of arrests seems to be larger for Sundays in case of vehicle thefts. On the other hand the number of accidents related to missing people increases only during working days. However, it is difficult to discern any patterns of behaviours in this case.